In [ ]:
from utils import *
from models import *

Loading the preprocessed files from dataset:
* b_de.npy, feature array representing information in the three different directions: frequency, temporal and spatial. 
* participant.npy array with the participant id.
* label.npy array with the corresponding state for each participant (0 distracted, 1 focus).

please feel the feat_d with the path to directory where the arrays are placed. 

In [ ]:
""" Load Files """

feat_d = 'phydaa'

feat = np.load(os.path.join(feat_d, 'b_de.npy'))
Label = np.load(os.path.join(feat_d, 'lab.npy'))
participant = np.load(os.path.join(feat_d, 'participant.npy'))

""" Model Training Parameters """

batch_size = 32
n_epoch = 500

participant_ind = True
participant_dep = False

In [ ]:
EEG = EEGDataset(label=Label, eeg=feat)
for p in np.unique(participant):
    # participant independent training 
    if participant_ind == True: 
        idx = np.argwhere(participant == p)[:, 0]
        np.random.shuffle(idx)
        if idx.shape[0]%32 == 1:
            idx = idx[0:-1]
        Test = Subset(EEG, idx)
        idx = np.argwhere(participant != p)[:, 0]
        np.random.shuffle(idx)
        if idx.shape[0]%32 == 1:
            idx = idx[0:-1]
        Train = Subset(EEG, idx)
    # participant dependent training
    elif participant_dep == True:
        idx = np.argwhere(participant == p)[:, 0]
        id_train, id_test = train_test_split(idx, test_size=0.2)

        idx = id_test
        np.random.shuffle(idx)
        if idx.shape[0]%32 == 1:
            idx = idx[0:-1]
        Test = Subset(EEG, idx)

        idx = id_train
        idx = np.argwhere(participant != p)[:, 0]
        np.random.shuffle(idx)
        if idx.shape[0]%32 == 1:
            idx = idx[0:-1]
        Train = Subset(EEG, idx)
        
    Trainloader = DataLoader(Train, batch_size=batch_size, shuffle=False)
    Testloader = DataLoader(Test, batch_size=batch_size, shuffle=False)

    wd = 1e-3
    name =  'Transformer_participant_'+str(p)+'_wd_'+str(wd)+'__'+feat_d+'__db'
    net = MultiAttention().cuda()
    #net = tot_RNN().cuda()
    #name =  'RNN_participant_'+str(p)+'_wd_'+str(wd)
    
    lr = 0.01
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=1-1e-2)

    writer = SummaryWriter(log_dir='runs/'+name)

    score = []
    print("training ", name)
    for epoch in range(n_epoch):
        running_loss = 0.0
        evaluation = []
        for i, data in enumerate(Trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            #inputs += gen_noise_tensor(inputs.shape)

            del data
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(torch.float32).cuda())
            loss = torch.nn.functional.cross_entropy(outputs, labels.to(torch.long).cuda())
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
            mean = num_of_true/labels.shape[0]
            running_loss += loss.item()
            evaluation.append(mean)

        running_loss = running_loss / (i + 1)
        running_acc = sum(evaluation) / len(evaluation)
        writer.add_scalar('Loss/train', running_loss, epoch)
        writer.add_scalar('Accuracy/train', running_acc, epoch)

        validation_loss = 0.0
        validation_acc = 0.0
        evaluation = []
        for i, data in enumerate(Testloader, 0):
            input_img, labels = data
            del data
            input_img = input_img.to(torch.float32)
            if True:
                input_img = input_img.cuda()
            outputs = net(input_img)
            loss = torch.nn.functional.cross_entropy(outputs, labels.cuda())
            validation_loss += loss.item()

            _, predicted = torch.max(outputs.cpu().data, 1)
            num_of_true = torch.sum(predicted == labels).numpy()

            evaluation.append(num_of_true/labels.shape[0])

        validation_loss = validation_loss / (i + 1)
        validation_acc = sum(evaluation) / len(evaluation)
        writer.add_scalar('Loss/test', validation_loss, epoch)
        writer.add_scalar('Accuracy/test', validation_acc, epoch)
        scheduler.step()
        writer.add_scalar('Misc/learning Rate', scheduler.get_lr()[0], epoch)
        score.append((running_loss, running_acc, validation_loss, validation_acc))
        print("Epoch {} \t---\tLoss {:.4f}, Accuracy {:.4f}\t---\tVal-Loss {:.4f}, Val-Accuracy {:.4f}" .format(epoch+1, running_loss, running_acc, validation_loss, validation_acc))
    score = np.asarray(score)
    np.save('t_dep/res/'+feat_d+'_db__res_sub_'+str(p), score)